In [3]:
import os
import pandas as pd
import re

# Especifica la carpeta donde se encuentran los archivos CSV
carpeta = 'GPA-Data-ElectroDunas'

# Lista para almacenar DataFrames individuales de cada archivo CSV
dataframes = []

# Itera sobre los archivos en la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith('.csv') and archivo.startswith('DATOSCLIENTE'):
        ruta_archivo = os.path.join(carpeta, archivo)
        
        # Utiliza expresiones regulares para extraer el número del cliente del nombre del archivo
        numero_cliente_match = re.search(r'\d+', archivo)
        if numero_cliente_match:
            numero_cliente = int(numero_cliente_match.group())
        else:
            numero_cliente = None  # Manejar el caso en el que no se encuentra un número
        
        # Lee el archivo CSV y almacena los datos en un DataFrame
        df = pd.read_csv(ruta_archivo)
        
        # Agrega una columna 'Cliente' al DataFrame
        df['Cliente'] = f'Cliente {numero_cliente}'
        
        # Agrega el DataFrame a la lista
        dataframes.append(df)

# Concatena todos los DataFrames en uno solo
df_final = pd.concat(dataframes, ignore_index=True)

# Convierte la columna 'Fecha' a tipo datetime
df_final['Fecha'] = pd.to_datetime(df_final['Fecha'])

# Crea nuevas columnas
df_final['Año'] = df_final['Fecha'].dt.year
df_final['Mes'] = df_final['Fecha'].dt.month
df_final['Trimestre'] = df_final['Fecha'].dt.quarter
df_final['Dia'] = df_final['Fecha'].dt.day
df_final['Hora'] = df_final['Fecha'].dt.hour
df_final['Dia_Semana'] = df_final['Fecha'].dt.day_name()

df_final.head()

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'GPA-Data-ElectroDunas'

In [34]:
## Max, Min, Desvest
max_active_energy = df_final['Active_energy'].max()
min_active_energy = df_final['Active_energy'].min()
std_active_energy = df_final['Active_energy'].std()

print(f'Max Active Energy: {max_active_energy}')
print(f'Min Active Energy: {min_active_energy}')
print(f'Standard Deviation of Active Energy: {std_active_energy}')

Max Active Energy: 14.622644283216186
Min Active Energy: -1.329018447895005
Standard Deviation of Active Energy: 1.7187798079929433


In [35]:
## Picos de Consumo
## Calcular la media y la desviación estándar de 'Active_energy'
media_consumo = df_final['Active_energy'].mean()
desviacion_estandar_consumo = df_final['Active_energy'].std()

## Definir un múltiplo de la desviación estándar como umbral para identificar picos positivos
umbral_picos_positivos = 2  

## Filtrar las filas que superan el umbral para identificar picos de consumo positivos
pico_positivo = df_final[df_final['Active_energy'] > media_consumo + umbral_picos_positivos * desviacion_estandar_consumo]

pico_positivo.shape

(28839, 12)

In [36]:
## Día de la semana con mayor consumo
dia_max_consumo = df_final.loc[df_final['Active_energy'].idxmax(), 'Dia_Semana']
print(f'Día de la semana de mayor consumo: {dia_max_consumo}')

Día de la semana de mayor consumo: Friday


In [37]:
## Mes con mayor consumo
mes_max_consumo = df_final.groupby('Mes')['Active_energy'].sum().idxmax()
print(f'Mes de mayor consumo: {mes_max_consumo}')

Mes de mayor consumo: 3


In [38]:
# Hora y Día de la semana con mayor consumo
hora_dia_max_consumo = df_final.loc[df_final['Active_energy'].idxmax(), ['Hora', 'Dia_Semana']]
print(f'Hora y día de la semana de mayor consumo:')
print(hora_dia_max_consumo)

Hora y día de la semana de mayor consumo:
Hora               1
Dia_Semana    Friday
Name: 374429, dtype: object


In [39]:
## Valores Negativos
valores_negativos = df_final[df_final['Active_energy'] < 0]
valores_negativos

,Fecha,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,Cliente,Año,Mes,Trimestre,Dia,Hora,Dia_Semana
133780,2021-11-09 15:00:00,-0.381904,0.382493,2077.430688,2093.655684,Cliente 17,2021,11,4,9,15,Tuesday
133781,2021-11-09 16:00:00,-0.408740,0.469523,2036.334551,2093.205889,Cliente 17,2021,11,4,9,16,Tuesday
133789,2021-11-10 00:00:00,-0.187803,0.392220,1950.083873,1951.344805,Cliente 17,2021,11,4,10,0,Wednesday
133790,2021-11-10 01:00:00,-0.455174,0.459796,1931.903407,1858.665907,Cliente 17,2021,11,4,10,1,Wednesday
133791,2021-11-10 02:00:00,-0.338941,0.390148,1910.706034,2064.042938,Cliente 17,2021,11,4,10,2,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...
216302,2022-05-09 12:00:00,-0.457582,0.200000,2069.593516,2077.851435,Cliente 20,2022,5,2,9,12,Monday
216303,2022-05-09 13:00:00,-0.458964,1.484933,2078.795633,2073.401393,Cliente 20,2022,5,2,9,13,Monday
216304,2022-05-09 14:00:00,-0.427294,0.231000,2071.330458,2079.444732,Cliente 20,2022,5,2,9,14,Monday
216305,2022-05-09 15:00:00,-0.455775,0.200000,2039.538550,2088.843514,Cliente 20,2022,5,2,9,15,Monday


In [40]:
## Completitud

## Total de filas antes de cambios
total_filas_antes = len(df_final)

## Verificar si hay valores nulos
completitud_sin_nulos = df_final['Active_energy'].notnull().all()

## Verificar si hay valores no numéricos
completitud_numerica = pd.to_numeric(df_final['Active_energy'], errors='coerce').notnull().all()

# Imprimir resultados
print(f'Completitud (sin valores nulos): {completitud_sin_nulos}')
print(f'Completitud (sin valores no numéricos): {completitud_numerica}')

Completitud (sin valores nulos): True
Completitud (sin valores no numéricos): True


In [44]:
## Clientes que dejan de reportar
clientes_faltantes = []

# Agrupar por cliente y fecha, contar las observaciones y verificar si hay algún día u hora faltante
for cliente, sub_df in df_final.groupby('Cliente'):
    fechas_esperadas = pd.date_range(sub_df['Fecha'].min(), sub_df['Fecha'].max(), freq='H' if 'Hora' in sub_df.columns else 'D')
    fechas_reales = sub_df['Fecha']
    
    if not fechas_esperadas.equals(fechas_reales):
        clientes_faltantes.append(cliente)

print('Clientes con días u horas faltantes:')
print(clientes_faltantes)
conteo_clientes_faltantes = len(clientes_faltantes)
print(f'Número de clientes con días u horas faltantes: {conteo_clientes_faltantes}')

Clientes con días u horas faltantes:
['Cliente 1', 'Cliente 10', 'Cliente 11', 'Cliente 12', 'Cliente 13', 'Cliente 14', 'Cliente 15', 'Cliente 16', 'Cliente 17', 'Cliente 18', 'Cliente 19', 'Cliente 2', 'Cliente 20', 'Cliente 21', 'Cliente 22', 'Cliente 23', 'Cliente 24', 'Cliente 25', 'Cliente 26', 'Cliente 27', 'Cliente 28', 'Cliente 29', 'Cliente 3', 'Cliente 30', 'Cliente 4', 'Cliente 5', 'Cliente 6', 'Cliente 7', 'Cliente 8', 'Cliente 9']
Número de clientes con días u horas faltantes: 30


In [52]:
## Verificar si todos los días se reportan mediciones
clientes_dias_sin_reporte = []

for cliente, sub_df in df_final.groupby('Cliente'):
    fechas_esperadas_dia = pd.date_range(sub_df['Fecha'].min(), sub_df['Fecha'].max(), freq='D')
    fechas_reales_dia = sub_df['Fecha'].dt.floor('D')
    
    if not fechas_esperadas_dia.equals(fechas_reales_dia):
        clientes_dias_sin_reporte.append(cliente)

print('Clientes con días sin reporte:')
print(clientes_dias_sin_reporte)
conteo_clientes_dias = len(clientes_dias_sin_reporte)
print(f'Número de clientes con días faltantes: {conteo_clientes_dias}')

Clientes con días sin reporte:
['Cliente 1', 'Cliente 10', 'Cliente 11', 'Cliente 12', 'Cliente 13', 'Cliente 14', 'Cliente 15', 'Cliente 16', 'Cliente 17', 'Cliente 18', 'Cliente 19', 'Cliente 2', 'Cliente 20', 'Cliente 21', 'Cliente 22', 'Cliente 23', 'Cliente 24', 'Cliente 25', 'Cliente 26', 'Cliente 27', 'Cliente 28', 'Cliente 29', 'Cliente 3', 'Cliente 30', 'Cliente 4', 'Cliente 5', 'Cliente 6', 'Cliente 7', 'Cliente 8', 'Cliente 9']
Número de clientes con días faltantes: 30


In [54]:
## Verificar si todas las horas se reportan mediciones
clientes_horas_sin_medicion = []

for cliente, sub_df in df_final.groupby('Cliente'):
    fechas_esperadas_hora = pd.date_range(sub_df['Fecha'].min(), sub_df['Fecha'].max(), freq='H')
    fechas_reales_hora = sub_df['Fecha'].dt.floor('H')
    
    if not fechas_esperadas_hora.equals(fechas_reales_hora):
        clientes_horas_sin_medicion.append(cliente)

print('Clientes con horas sin medición:')
print(clientes_horas_sin_medicion)
conteo_clientes_horas = len(clientes_horas_sin_medicion)
print(f'Número de clientes con días faltantes: {conteo_clientes_horas}')

Clientes con horas sin medición:
['Cliente 1', 'Cliente 10', 'Cliente 11', 'Cliente 12', 'Cliente 13', 'Cliente 14', 'Cliente 15', 'Cliente 16', 'Cliente 17', 'Cliente 18', 'Cliente 19', 'Cliente 2', 'Cliente 20', 'Cliente 21', 'Cliente 22', 'Cliente 23', 'Cliente 24', 'Cliente 25', 'Cliente 26', 'Cliente 27', 'Cliente 28', 'Cliente 29', 'Cliente 3', 'Cliente 30', 'Cliente 4', 'Cliente 5', 'Cliente 6', 'Cliente 7', 'Cliente 8', 'Cliente 9']
Número de clientes con días faltantes: 30


In [1]:
import pandas as pd

# Variables de interés
variables_interes = ['Active_energy', 'Voltaje_FA', 'Voltaje_FC']

# Función para calcular las estadísticas deseadas
def calcular_estadisticas(variable):
    return pd.Series({
        'Mínimo': df_final[variable].min(),
        'Primer cuartil': df_final[variable].quantile(0.25),
        'Promedio': df_final[variable].mean(),
        'Mediana': df_final[variable].median(),
        'Tercer cuartil': df_final[variable].quantile(0.75),
        'Máximo': df_final[variable].max(),
        'Desviación': df_final[variable].std()
    }, name=variable)

# Crear un DataFrame con las estadísticas para cada variable
tabla_estadisticas = pd.DataFrame([calcular_estadisticas(variable) for variable in variables_interes])

# Imprimir la tabla
print(tabla_estadisticas)

NameError: name 'df_final' is not defined